In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [3]:
# 续租买断尾款 为续租总租金，续租买断尾款其实为0
# 首租订单，首租买断尾款为续租总租金，实际中，续租总租金 大于 首租买断尾款
# all_money ：续租的已完成的话，为首租+续租总租金；续租的未完成的话，为首租的总租金  'A202308262317036','A202207050851373'
#     , tprm.purchase_amount 采购金额
# tprm.order_id, tprm.order_number, tprm.order_create_time 下单日期

sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,tod.product_name

-- 2025.6.23
,cc.name as channel_name_cc         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,pa.type
,om.order_type
,om.all_money 首租总租金
,tod.actual_money 首租买断价
,(tod.actual_money-om.all_money) 首租买断尾款
,ym.name
,too.other_money as 违约金 -- 支付完成的违约金+赔偿金+滞纳金


,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result

,toou.discount_money, om.has_actual, tod.new_actual_money, tod.sku_attributes
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id

left join (
SELECT t.order_id,sum(t.money) as other_money  -- 支付完成的违约金+赔偿金+滞纳金
from db_digua_business.t_order_other t 
where t.`status`=2 GROUP BY 1 ) too on too.order_id=om.id

-- 2025.6.23
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
left join db_digua_business.t_order_out toou on toou.order_id=tprm.order_id


;
'''

df_zhys = query(sql)
# (27787, 62)
df_zhys.shape
df_zhys[:1]

,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,...,首租买断尾款,name,违约金,tips,is_vip,status_result,discount_money,has_actual,new_actual_money,sku_attributes
0,1,8924,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,...,-317.2,维客自营账户,0.0,None,None,None,0.0,1,8176.92,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu..."


In [4]:
sql_zf = '''
SELECT 
h.order_number, h.type,h.amount, h.plan_id
from db_rent.ya_merchant_account h 
where h.financial_type in (2, 3, 4, 5, 6, 7, 8, 9, 10, 11) 
and h.is_transfer = 0 
'''
df_zf = query(sql_zf)
df_zf.loc[:, '支出'] = np.where(df_zf.type==1, df_zf.amount, 0)
df_zf.loc[:, '收入'] = np.where(df_zf.type==0, df_zf.amount, 0)
df_zf.loc[:, '其它费用'] = df_zf.支出-df_zf.收入
df_zf_group = df_zf.groupby('order_number').agg({'其它费用': 'sum'})

In [5]:
df_xz_concat = pd.read_excel('F:/租后表分析数据_ld/tmp2.xlsx')
tmp2 = df_xz_concat.groupby(['order_number']).agg({'is_relet':'max', '当前应付租金':'sum', '实付金额':'sum'}).reset_index()
df_zhys = df_zhys.merge(tmp2, on = 'order_number', how = 'left')


In [6]:
sql_stages = '''
select
order_id, sum(money) 租金
from db_rent.ya_merchant_order_stages
group by order_id
'''
df_stages = query(sql_stages)

In [7]:
# 南宁微美汇
# 广州捷瑞环保科技
# 山东易路人店
# 广州康基店
# 绿能智联科技
# LK雷铠科技

# 归还/买断/续租
# 订单量	总收入	总采购成本	总第三方费用
# 归还：总收入 = 违约金+租金+残值收入
# 买断：总收入 = 违约金+租金+买断尾款
# 续租：总收入 = 违约金+租金

df_zhys = df_zhys.merge(df_zf_group[['其它费用']], on='order_number', how='left')
df_zhys = df_zhys.merge(df_zf[['order_number', 'plan_id']], on='order_number', how='left')
df_zhys = df_zhys[df_zhys.plan_id!=220]
df_zhys.loc[:,"租赁方案"]=np.where(df_zhys["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df_zhys = df_zhys[df_zhys.purchase_amount>0]
df_zhys.loc[:, 'discount_money'] = df_zhys.discount_money.fillna(0)
df_zhys.loc[:, '买断尾款'] = df_zhys.new_actual_money-df_zhys.all_rental-df_zhys.discount_money
df_zhys.loc[:, '违约金'] = df_zhys.违约金.fillna(0)
df_zhys.loc[:, '总收入'] = np.where((df_zhys.订单状态值==8)&(df_zhys.has_actual==0), df_zhys.违约金+df_zhys.实付金额+df_zhys.paid_buyout_balance_payment,
                                    np.where((df_zhys.订单状态值==8)&(df_zhys.has_actual==1), df_zhys.违约金+df_zhys.实付金额+df_zhys.买断尾款,
                                                np.where(df_zhys.is_relet>=1, df_zhys.违约金+df_zhys.实付金额, 0)))
df_zhys.loc[:, '总收入'] = np.where(df_zhys.租赁方案=='租完即送', df_zhys.all_rental-df_zhys.discount_money, df_zhys.总收入)
df_zhys.loc[:, '总收入'] = np.where(df_zhys.总收入<df_zhys.purchase_amount, df_zhys.实付金额+df_zhys.paid_buyout_balance_payment, df_zhys.总收入)
df_zhys = df_zhys.drop_duplicates(subset=['order_number'])

conditions_1 = (df_zhys.name.isin(['南宁微美汇','广州捷瑞环保科技','山东易路人店','广州康基店','绿能智联科技','LK雷铠科技']))
conditions_2 = (df_zhys.search_time >= '2024-05') & (df_zhys.search_time <= '2025-07')

df_zhys_gh = df_zhys[(df_zhys.订单状态值==8)&(df_zhys.has_actual==0)]
df_zhys_group_gh = df_zhys_gh[conditions_1 & conditions_2].groupby(['name']).agg({'order_number':'count', '总收入': 'sum','purchase_amount':'sum','其它费用':'sum'})


df_zhys_md = df_zhys[(df_zhys.订单状态值==8)&(df_zhys.has_actual==1)]
df_zhys_group_md = df_zhys_md[conditions_1 & conditions_2].groupby(['name']).agg({'order_number':'count', '总收入': 'sum','purchase_amount':'sum','其它费用':'sum'})


df_zhys_xz = df_zhys[(df_zhys.is_relet>=1)&(df_zhys.订单状态值!=8)]
df_zhys_xz = df_zhys_xz.merge(df_stages, on='order_id', how='left')
df_zhys_xz.loc[:, '总收入'] = df_zhys_xz.违约金+df_zhys_xz.租金
df_zhys_group_xz = df_zhys_xz[(df_zhys_xz.name.isin(['南宁微美汇','广州捷瑞环保科技','山东易路人店','广州康基店','绿能智联科技','LK雷铠科技'])) & (df_zhys_xz.search_time >= '2024-05') & (df_zhys_xz.search_time <= '2025-07')].groupby(['name']).agg({'order_number':'count', '总收入': 'sum','purchase_amount':'sum','其它费用':'sum'})
df_zhys_group_xz

,order_number,总收入,purchase_amount,其它费用
name,,,,
LK雷铠科技,15,183442.87,105507.0,10315.29
南宁微美汇,107,1390719.02,775134.0,89046.07
山东易路人店,94,1089573.02,632632.0,62905.22
广州康基店,78,953801.04,552489.0,55379.27
广州捷瑞环保科技,113,1447242.57,825295.0,87703.54
绿能智联科技,34,400903.19,234577.0,22575.10


In [8]:
# 南宁市微美汇日用品商行
# 广州捷瑞环保科技有限公司
# 山东易路人自动化设备有限公司
# 广州康基贸易有限公司
# 深圳市绿能智联科技有限公司
# 深圳市雷凯光电科技有限公司
pd.concat([df_zhys_group_gh, df_zhys_group_md, df_zhys_group_xz], axis=1)

,order_number,总收入,purchase_amount,其它费用,order_number,总收入,purchase_amount,其它费用,order_number,总收入,purchase_amount,其它费用
name,,,,,,,,,,,,
LK雷铠科技,5,48588.12,36021.0,2864.56,72,759051.44,501333.0,73934.26,15,183442.87,105507.0,10315.29
南宁微美汇,9,85366.81,58464.0,5235.53,58,706537.03,426155.0,66561.68,107,1390719.02,775134.0,89046.07
山东易路人店,10,76125.59,61474.0,4787.23,68,696163.76,451534.0,67759.51,94,1089573.02,632632.0,62905.22
广州康基店,10,79010.03,60021.0,4814.20,76,808927.69,523294.0,78439.35,78,953801.04,552489.0,55379.27
广州捷瑞环保科技,12,115392.17,83894.0,6807.29,63,746027.89,469203.0,72196.95,113,1447242.57,825295.0,87703.54
绿能智联科技,15,119940.92,90247.0,7485.14,54,572497.36,368103.0,55251.77,34,400903.19,234577.0,22575.10
